In [174]:
import pandas as pd
from datasets import load_dataset, Dataset
import uuid
from copy import deepcopy

In [175]:
file_path = "search_qa"
dataset = load_dataset(file_path, 'raw_jeopardy')['train'].to_pandas()
dataset['value'] = dataset['value'].fillna('$0')
dataset = dataset.dropna()
metadata = dataset[['air_date', 'category', 'value', 'round', 'show_number']]
dataset = dataset.drop(['air_date', 'category', 'value', 'round', 'show_number'], axis=1)

## dataset에 query_id 추가

In [176]:
dataset = pd.Series([str(uuid.uuid4()) for _ in range(len(dataset))], name='query_id').to_frame().join(dataset)

## preprocessing corpus
task: retrieval gt를 만들기 위한 corpus(metadata가 없음)와 ingestion을 위한 corpus(metadata가 있음)를 만들어야함

In [177]:
corpus = dataset['search_results'].apply(pd.Series)
corpus = corpus.drop(['related_links', 'titles', 'urls'], axis=1)

1. query하나당 똑같은 snippet이 나오지 체크
2. 동일 쿼리 안에 중복되는 snippet 제거

In [178]:
corpus['snippets'] = corpus['snippets'].apply(lambda x: list(set(x.tolist())))

#### test code
중복 snippet이 있는지 체크해주는 test코드

In [179]:
for i in corpus['snippets']:
    test = set(i)
    if len(test) != len(i):
        raise ValueError("There is same snippets in corpus['snippets']")

corpus를 metadata와 분리 어차피 그룹바이하고나서 meta_data는 붙여바로면 됌<br>
corpus는 query_id/snippet/air_date/category/value/round/show_number형태임

In [180]:
corpus = pd.concat([dataset['query_id'], corpus, metadata], axis=1)  # metadata 있는 corpus용 corpus

#### test code
corpus가 null값이 있는지 체크

In [181]:
check_corpus_isnull = corpus.isnull().sum()
print(check_corpus_isnull)
# assert check_corpus_isnull['snippets'] == 0  # corpus에 null값이 있는지 확인(특히 snippets에 null값이 있는지 확인)

query_id       0
snippets       0
air_date       0
category       0
value          0
round          0
show_number    0
dtype: int64


결과:<br>
모두 null값 없음

## explode해서 flatten함.
#### test code

In [182]:
for_checking_lost_snippets = 0
for i in corpus['snippets']:
    for_checking_lost_snippets += len(i)
    # if None in i:
    #     raise ValueError("There is None in corpus['snippets']")
    # TODO: 이것을 확인하기 위해서 corpus의 query_id와 원본 query_id를 비교해보자 실험

정보:
- snippet 자체에서 null값이 있는것을 확인했음
- 한 query안에 모두 null이라 query 자체가 날라가는것도 있음

In [183]:
corpus = corpus.explode(['snippets']).reset_index(drop=True)

#### test code

In [184]:
# Check if snippet are lost or not.
assert len(corpus) == for_checking_lost_snippets

check_explode_corpus_isnull = corpus.isnull().sum() 
print(f"explode해서 생긴 null값 체크 : {check_explode_corpus_isnull}")# corpus에 null값이 있는지 확인(특히 snippets에 null값이 있는지 확인)
# assert check_explod_corpus_isnull['snippets'] == 0  # corpus에 null값이 있는지 확인(특히 snippets에 null값이 있는지 확인)

# TODO: explode가 문제인것을 확인 -> 그렇다면 원래 snippets이 null값이었던것인가? -> 원래 snippet이 여러개였던 녀석 존나 많음

# TODO: dropna가 문제인지 체크하기 위해서 따로 빼놓음
corpus = corpus.dropna()

explode해서 생긴 null값 체크 : query_id            0
snippets       116988
air_date            0
category            0
value               0
round               0
show_number         0
dtype: int64


## dupliacte corpus 제거

In [185]:
gt_corpus = corpus.drop(['air_date', 'category', 'value', 'round', 'show_number'], axis=1)  # gt를 만들기 위한 corpus
copy_corpus = deepcopy(corpus)
copy_corpus = copy_corpus.dropna()

#### test code
duplicate snippet이 merge할때 null이 생기는것과 동일한가?

In [186]:
check_isduplicate_before_drop_duplicates = copy_corpus['snippets'].duplicated().sum()
print(check_isduplicate_before_drop_duplicates)

160754


In [187]:
copy_corpus = copy_corpus.drop_duplicates(subset='snippets', keep='first')

#### test code
corpus에 null값이 있는지 확인(특히 snippets에 null값이 있는지 확인)

In [188]:
check_isnull_after_drop_duplicates = copy_corpus.isnull().sum()
print(check_isnull_after_drop_duplicates)# 중복된 snippet은 제거된 상태, 즉 순수한 corpus 추출
assert check_isnull_after_drop_duplicates['snippets'] == 0  # corpus에 null값이 있는지 확인(특히 snippets에 null값이 있는지 확인)

query_id       0
snippets       0
air_date       0
category       0
value          0
round          0
show_number    0
dtype: int64


#### test code
여기서 snippet 외의 none값이 발생되어서 원래 있어야할 snippet과 doc_id가 사라져 매칭이 안되는 억까 ->

In [189]:
check_isnull_after_drop_duplicates = copy_corpus.isnull().sum()  # 중복된 snippet은 제거된 상태, 즉 순수한 corpus 추출
assert check_isnull_after_drop_duplicates['snippets'] == 0  # corpus에 null값이 있는지 확인(특히 snippets에 null값이 있는지 확인)

In [190]:
copy_corpus = copy_corpus.dropna()

중복된 snippet은 제거된 상태, 즉 순수한 corpus 추출, doc_id생성

# Doc id 부여

In [191]:
def __make_doc_id(row):
    return str(uuid.uuid4())

In [192]:
copy_corpus[copy_corpus.snippets == '']

,query_id,snippets,air_date,category,value,round,show_number
4954,aa6c6277-d923-4c6a-8ac8-86e0237b6900,,2010-07-06,RHYMES WITH SMART,$200,Jeopardy!,5957


In [193]:
copy_corpus[copy_corpus.snippets == ' ']

,query_id,snippets,air_date,category,value,round,show_number
6153700,565ff775-f032-4342-bc39-2f4a7f911581,,2005-11-23,BEST FOREIGN LANGUAGE FILM,$1600,Double Jeopardy!,4878


In [194]:
# 빈칸으로 들어가있는 예상치 못했던 ㅈ같은 snippet들 제거
before = len(copy_corpus)
copy_corpus = copy_corpus[~(copy_corpus.snippets == ' ')]
copy_corpus = copy_corpus[~(copy_corpus.snippets == '')]
after = len(copy_corpus)
assert before - after == 2

In [195]:
copy_corpus['doc_id'] = copy_corpus.apply(__make_doc_id, axis=1)
complete_corpus = deepcopy(copy_corpus)  # corpus로 hugging에 push할것(query_id/snippet/air_date/category/value/round/show_number/doc_id)

순수한 copy_corpus를 가지고 gt를 만들기 위해 원본 corpus를 이용해서 query_id를 매칭시켜 gt를 부여

In [196]:
gt_copy_corpus = copy_corpus.drop(['air_'
                                   'date', 'category', 'value', 'round', 'show_number'], axis=1)

In [197]:
check_isnull_merged_df = gt_copy_corpus.isnull().sum()
print(check_isnull_merged_df)

query_id    0
snippets    0
doc_id      0
dtype: int64


## Merge해서 qa_data의 retrieval gt 만들기

In [198]:
merged_df = gt_corpus.merge(gt_copy_corpus, on='snippets', how='inner').drop(['query_id_y'], axis=1).rename(
    columns={'query_id_x': 'query_id'})

#### test code
여기서 null 값이 있다면 snippets에 null값이 있었다는 뜻임 -> retrieval gt fetch할때 문제

In [199]:
check_isnull_merged_df = merged_df.isnull().sum()
print(check_isnull_merged_df)

query_id    0
snippets    0
doc_id      0
dtype: int64


merge한 다음 groupby해서 corpus 만들기 -> groupby했을때 원본데이터의 shape이랑 같아야함.<br>
원본 qa_data와의 shape를 같게 맞춰주기 위해서 query_id를 groupby해서 retrieval_gt리스트들을 만들어줌

In [200]:
create_gt = (merged_df.groupby('query_id', as_index=False).agg({'snippets': lambda x: list(x),
                                                                'doc_id': lambda x: list(x)}))

for check_null in create_gt['snippets']:
    if None in check_null:
        raise ValueError("There is None in create_gt['snippets']")
for check_null in create_gt['doc_id']:
    if None in check_null:
        raise ValueError("There is None in create_gt['doc_id']")

Remove none gt rows because of corpus can't retrieve none gt and removed query in preprocessing process.

In [201]:
mask = dataset['query_id'].isin(create_gt['query_id'])
dataset = dataset[mask]
assert len(create_gt) == len(dataset)  # 원본 qa_data와의 shape를 같게 맞춰주기 위해서 query_id를 groupby해서 retrieval_gt리스트들을 만들어줌

#### 존내 오래걸리는 test code
qa_data의 retrieval gt와 corpus의 doc_id가 매칭이 되는지 확인

In [202]:
# qa_data의 doc_id가 corpus의 doc_id에 있는지 체크 -> 여기서 없으면 이상한게 지금 캐시에 저장이 되었는지 preprcessing과정에서 아래 과정처럼 확인
# qa = create_gt['doc_id'].tolist()
# 
# for i in qa:
#     for j in i:
#         if (complete_corpus['doc_id'] == j).any() == False:
#             raise ValueError(f"doc_id {j} is not in corpus")

#### Hakey하게 length만 비교해서 qa_data가 groupby하는 과정에서 doc_id(retrieval gt)가 누락되는것을 확인하는 test 코드

In [203]:
test_qa_by_len = create_gt['doc_id']
test_qa_by_len = len(test_qa_by_len.explode('doc_id'))
print(test_qa_by_len)
print(len(merged_df))
assert test_qa_by_len == len(merged_df)

14276991
14276991


#### qa_data에 doc_id(gt) concat

In [204]:
from sklearn.model_selection import train_test_split
qa_dataset = pd.concat([dataset, create_gt['doc_id']], axis=1).rename(columns={'doc_id': 'retrieval_gt'})

check_qadata_isnull = qa_dataset.isnull().sum()  # qa_dataset에 null값이 있는지 확인

qa_dataset_train, qa_dataset_test = train_test_split(qa_dataset, test_size=0.2, shuffle=False)

# 최종 오의

In [205]:
qa_dataset_train = Dataset.from_pandas(qa_dataset_train)
qa_dataset_test = Dataset.from_pandas(qa_dataset_test)
complete_corpus = Dataset.from_pandas(complete_corpus)

qa_dataset_train.push_to_hub("NomaDamas/search_qa_split", 'qa_data', split='train')
qa_dataset_test.push_to_hub("NomaDamas/search_qa_split", 'qa_data', split='test')
complete_corpus.push_to_hub("NomaDamas/search_qa_split", 'corpus')

Pushing dataset shards to the dataset hub:   0%|          | 0/14 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1087 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1087 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1087 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1087 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1087 [00:00<?, ?ba/s]